## Metaclasses and allocation

### Instance creation
* when creating an object, the \_\_init\_\_ method will be invoke. Before this method is invoked, the object has already been created, and passed to this method as 'self'
* during initialization process, the attributes of the object are assigned/modified by object.\_\_setattr\_\_ method
* \_\_init\_\_ method doesn't return anything. It just mutate what the self object was given
* since \_\_init\_\_ method doesn't create new object, which method does this?
  + \_\_new\_\_
  + usually \_\_new\_\_ is inherited from Object. This method allocates object which is then passed to \_\_init\_\_ as self
  + \_\_new\_\_ is an implicit static method having cls as its first argument
  + cls is the class of the object which will be allocated. In the code example, this will be ChessCoordinate class object
  + this method must use the cls as the first argument with other arguments forwarded from constructor calls, to create an object and and return the object
    + example code shows that the arguments provided when instantialte the ChessCoordinate are received from \_\_new\_\_()
    + the \_\_new\_\_() method only use cls to create the object with an id, and then return the object
    + the object and other arguments of object instantiation ('d', and 4) are forwarded to \_\_init\_\_ as file and rank, respectively
  

In [16]:
class ChessCoordinate:
    
    def __new__(cls, *args, **kwargs):
        print(f"cls = {cls.__name__}")
        print(f"args = {args!r}")
        print(f"kwargs = {kwargs!r}")
        obj = object.__new__(cls)
        print(f"id(obj) = {id(obj)}")
        return obj
    
    def __init__(self, file, rank):
        
        print(f"id(self) = {id(self)}")
        
        if len(file) != 1:
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"is out of range."
            )
        
        self._file = file
        self._rank = rank
        
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [17]:
white_queen = ChessCoordinate("d", 4)
print(white_queen)
        

cls = ChessCoordinate
args = ('d', 4)
kwargs = {}
id(obj) = 1854128552736
id(self) = 1854128552736
d4


In [23]:
class ChessCoordinate:
    
    def __new__(cls, *args, **kwargs):        
        obj = object.__new__(cls)        
        return obj
    
    def __init__(self, file, rank):        
              
        if len(file) != 1:
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{type(self).__name__} component file {file!r} "
                f"is out of range."
            )
        
        self._file = file
        self._rank = rank
        
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [24]:
# code for measuring memory usage
import tracemalloc
tracemalloc.start()
white_queens = [ ChessCoordinate("d", 4) for _ in range(10000)]
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak_kb = peak / 10 **3
print(f"{peak_kb:.0f} KB")

1616 KB


### Customizing allocation
#### Interning
* re-using objects of equal value on-demand instead of creating new objects
* we can interning objects by overriding \_\_new\_\_
* when we create a large numbers of object, the memory usage is linearly increased
* for our use case, since there are only 64 positions on the chessboard, we only need 64 or 32 boards to record all the positions on the board.
+ to implement the interning, we move the code from \_\_init\_\_ to \_\_new\_\_ and put arguments of instantiation to \_\_new\_\_
  + we change the self to cls before object creation, and after ChessCoordinate object is created, assign the \_file and \_rank to object, rather than self in \_\_new\_\_
  + we created a dictionary in the class definition to store the ChessCoordinate instance using its file and rank combination as the key. We only create new ChessCoordinate instances corresponding to a new key and store the created object to the \_interned dictionary. Otherwise, we just retrieve the object from the dictionary and return it 
  + we then delete the \_\_init\_\_ method
* intering can reduce the memory usage by reducing the number of instances created
  + it can only used for immutable value types

In [25]:
class ChessCoordinate:
    
    _interned = {}
    
    def __new__(cls, file, rank):
        
        if len(file) != 1:
            raise ValueError(
                f"{cls.__name__} component file {file!r} "
                f"does not have a length of one."
            )
            
        if file not in "abcdefgh":
            raise ValueError(
                f"{cls.__name__} component file {file!r} "
                f"is out of range."
            )
        
        key = (file, rank)
        if key not in cls._interned:
            obj = object.__new__(cls) 
            obj._file = file
            obj._rank = rank
            cls._interned[key] = obj
        
        return cls._interned[key]
    
            
    @property
    def file(self):
        return self._file

    @property
    def rank(self):
        return self._rank

    def __repr__(self):
        return f"{type(self).__name__}(file={self.file}, rank={self.rank})"

    def __str__(self):
        return f"{self.file}{self.rank}"       

In [26]:
# code for measuring memory usage
import tracemalloc
tracemalloc.start()
white_queens = [ ChessCoordinate("d", 4) for _ in range(10000)]
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak_kb = peak / 10 **3
print(f"{peak_kb:.0f} KB")

98 KB


#### Summary of instance allocation and creation
* the special method \_\_new\_\_ allocates and return new instances
* the new instance is passed to \_\_init\_\_ as self
* the ultimat allocator is object.\_\_new\_\_(cls) that is responsible for allocating all instances. It must be passed the class of object to be allocated
* we can override \_\_new\_\_ in our own classes to customize allocation
  + the \_\_new\_\_(cls) method is implicitly static, which accept the class of the new instance as its first argument, and does not require classmethod or staticmethod decorators
* interning reuses existing objects of equal value (in our code example, the key of the instance represented by file and rank) to save memory usage 
  + this can be useful when certain values of immutable value types are very common, or when the domain of values is small and finite 

### Metaclasses and Class Creation

#### class creation and Metaclasses
* metaclasses
  + class of classes
  + type of any class object in Python is its metaclass, and the default metaclass is type, including type class itself
* when we define a class, we implicitly use object as the default base class, and type as the default metaclass. Specifically,

```python
class Widget:
    pass
``` 
can be explicitly expressed as 

```python
class Widget(object, metaclass=type):
    pass
```
* class definition process
  + when we define a class using class Widget,
    + the new class's metaclass creates an empty namespace dictionary
    + python runtime extracts the content of the class definition block and populates the dictionary
    + python runtime then hand it back to metaclass 
    + metaclass convert it into the class object that bound to class name (allocates class object)
  + in the following code, the name, bases, and namespace arguments contain information collected during execution of class definition, class attributes and method definitions inside the class block
  + by providing our own metaclass, we can customize these behaviors  
  + to use python code to describe the process:
  ```python
    class Widget:
        pass
    
    # the following are what happens:
    
    name = 'Widget'      # class name is Widget
    metaclass = type     # metaclass is the default, type
    bases = ()           # no base class other than the implicit object
    kwargs = {}          # no keyword args
    
    # metaclass create a new namespace, which behaviors like a dictionary
    namespace = metaclass.__prepare__(name, bases, **kwargs)
    # after this call, python runtime populates this namespace dictionary with entries based on
    # the contents of the class block in source code
    
    # metaclass's __new__ method is called to allocate the object
    Widget = metaclass.__new__(metaclass, name, bases, namespace, **kwargs)
    
    # then metaclass's __init__ method is called to initialize the class object
    metaclass.__init__(Widget, name, bases, namespace, **kwargs)
    
    ```    

In [27]:
class Widget:
    pass
w = Widget()

# metaclass of an instance is its class
print(w.__class__)
print(w.__class__.__class__)
print(w.__class__.__class__.__class__)

<class '__main__.Widget'>
<class 'type'>
<class 'type'>


#### code demo of the process
* the following code shows the entire process by defining our own metaclass: TracingMeta
* TracingMeta just print out the cls, name, namespace, bases, keyword args values during the new object creation process
* it inherits the default metaclass, type
* all the methods in TracingMeta just called the super(), which are the methods executed by type metaclass
* note that \_\_new\_\_ is an implicit class method, and we need to decorate \_\_prepare\_\_ by @classmethod

In [30]:
class TracingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        mcs:      metaclass itself, similar to 'self' in a normal class method signature
        name:     name of the metaclass
        bases:    a tuple of base classes, the ultimate object class is implicite
        **kwargs: an empty dictionary
        """
        print("TracingMeta.__prepare__(name, bases, **kwargs)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {kwargs = }")
        
        # returns a mapping type, here an empty dictionary as a namespace
        # assciated with the nascent class
        namespace = super().__prepare__(name, bases)
        print(f"-> {namespace = }")
        print()
        return namespace
    
    # create new class object
    def __new__(mcs, name, bases, namespace, **kwargs):
        """
        namespace: namespace mapping returned from __prepare__
        """
        print("TracingMeta.__new__(mcs, name, bases, namespace)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        
        # python runtime populated this dictionary with several entries
        # as it processed the class definition of Widget class.
        # including class attributes and methods defined in the class
        # it also adds module, which is the name of the file that contains this class
        # and fully qualified name, which is a gobal class name since it is in bulit-in
        print(f" {namespace = }")
        print(f" {kwargs = }")
        
        # create a new Widget class object
        # any changes we want to make to the namespace object must be made before this call
        # to change the contents of the class namespace after this call,the class object
        # must be manipulated directly
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"-> {cls = }")
        print()
        return cls
    
    # this method configure the class object
    # we could modify namespace by the method, before the class object is instantiated
    # we could modify the sequence of base classes, and allocate a different class entirely to cls
    def __init__(cls, name, bases, namespace, **kwargs):
        """
        this is an instance method of the metaclass. both __new__ and __prepare__ are class methods
        cls: Widget class object (one level less than TracingMeta class, the same way 'self' less than cls)
        """
        print("TracingMeta.__init__(cls, name, bases, namespace)")
        print(f" {cls = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {namespace = }")
        print(f" {kwargs = }")
        super().__init__(name, bases, namespace)
        print(f"-> {cls = }")
        print()  
    

In [31]:
class Widget(metaclass=TracingMeta):
    the_answer = 42
    
    def action(self, message):
        print(message)

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget', 'the_answer': 42, 'action': <function Widget.action at 0x000001AFB3616EF0>}
 kwargs = {}
-> cls = <class '__main__.Widget'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Widget'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget', 'the_answer': 42, 'action': <function Widget.action at 0x000001AFB3616EF0>}
 kwargs = {}
-> cls = <class '__main__.Widget'>



#### Which methods we should override
* \_\_prepare\_\_:
  + customize the type or initial value of the namespace mapping
  + default is to generate a regular empty dictionary for the namespace object
  + you only override this method if you need another mapping type
* usually it will be necessary to override either \_\_new\_\_ or \_\_init\_\_
  + only __new__ can make decisions before the new class is allocated
  + can configure the class object so that metaclasses are more composable

### Metaclass details

#### Metaclass keyword arguments
* any keyword arguments above the metaclass argument in the class definition will be fowarded to the 3 metaclass methods
```python
# more, keyword, and args will be forwarded to the 3 metaclass methods
class Widget(object, metaclass=type, more=1, keyword=2, args=3):
    pass
```

* using the keyward args can make metaclass as a class factory

In [32]:
class Reticulator(metaclass=TracingMeta, tension=496):
    cubic = True
    def reticulate(self, spline):
        print(spline)

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Reticulator'
 bases = ()
 kwargs = {'tension': 496}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Reticulator'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Reticulator', 'cubic': True, 'reticulate': <function Reticulator.reticulate at 0x000001AFB36167A0>}
 kwargs = {'tension': 496}
-> cls = <class '__main__.Reticulator'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Reticulator'>
 name = 'Reticulator'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Reticulator', 'cubic': True, 'reticulate': <function Reticulator.reticulate at 0x000001AFB36167A0>}
 kwargs = {'tension': 496}
-> cls = <class '__main__.Reticulator'>



In [43]:
class EntriesMeta(type):
    
    def __new__(mcs, name, bases, namespace, num_entries):
        print("EntriesMeta.__new__(mcs, name, bases, namespace, **kwargs)")
        print(f" {num_entries = }")
        namespace.update({chr(i): i for i in range(ord('a'), ord('a') + num_entries)})
        cls = super().__new__(mcs, name, bases, namespace)
        return cls        

In [44]:
class AtoZ(metaclass=EntriesMeta, num_entries=26):
    pass

EntriesMeta.__new__(mcs, name, bases, namespace, **kwargs)
 num_entries = 26


In [45]:
from pprint import pprint
pprint(dir(AtoZ))
AtoZ.f

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']


102

#### Metaclass method visibility
* other methods except for the three methods above in metaclass definition can be access as regular methods from the class objects created by class_name.method_name. These methods are called metamethods
* we can't access metamethods from the instances of the class, as shown in the code example
* metamethods are rarely used, except for \_\_call\_\_() method
* class methods defined in the class and its base classes will take the precedence over the class methods defined in its metaclass and metaclass's base class

In [46]:
class TracingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        mcs:      metaclass itself, similar to 'self' in a normal class method signature
        name:     name of the metaclass
        bases:    a tuple of base classes, the ultimate object class is implicite
        **kwargs: an empty dictionary
        """
        print("TracingMeta.__prepare__(name, bases, **kwargs)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {kwargs = }")
        
        # returns a mapping type, here an empty dictionary as a namespace
        # assciated with the nascent class
        namespace = super().__prepare__(name, bases)
        print(f"-> {namespace = }")
        print()
        return namespace
    
    # create new class object
    def __new__(mcs, name, bases, namespace, **kwargs):
        """
        namespace: namespace mapping returned from __prepare__
        """
        print("TracingMeta.__new__(mcs, name, bases, namespace)")
        print(f" {mcs = }")
        print(f" {name = }")
        print(f" {bases = }")
        
        # python runtime populated this dictionary with several entries
        # as it processed the class definition of Widget class.
        # including class attributes and methods defined in the class
        # it also adds module, which is the name of the file that contains this class
        # and fully qualified name, which is a gobal class name since it is in bulit-in
        print(f" {namespace = }")
        print(f" {kwargs = }")
        
        # create a new Widget class object
        # any changes we want to make to the namespace object must be made before this call
        # to change the contents of the class namespace after this call,the class object
        # must be manipulated directly
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"-> {cls = }")
        print()
        return cls
    
    # this method configure the class object
    # we could modify namespace by the method, before the class object is instantiated
    # we could modify the sequence of base classes, and allocate a different class entirely to cls
    def __init__(cls, name, bases, namespace, **kwargs):
        """
        this is an instance method of the metaclass. both __new__ and __prepare__ are class methods
        cls: Widget class object (one level less than TracingMeta class, the same way 'self' less than cls)
        """
        print("TracingMeta.__init__(cls, name, bases, namespace)")
        print(f" {cls = }")
        print(f" {name = }")
        print(f" {bases = }")
        print(f" {namespace = }")
        print(f" {kwargs = }")
        super().__init__(name, bases, namespace)
        print(f"-> {cls = }")
        print()  
    
    def metamethod(cls):
        print("TracingMeta.metamethod(cls)")
        print(f" {cls = }")

In [47]:
class Widget(metaclass=TracingMeta):
    pass

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(mcs, name, bases, namespace)
 mcs = <class '__main__.TracingMeta'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget'}
 kwargs = {}
-> cls = <class '__main__.Widget'>

TracingMeta.__init__(cls, name, bases, namespace)
 cls = <class '__main__.Widget'>
 name = 'Widget'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'Widget'}
 kwargs = {}
-> cls = <class '__main__.Widget'>



In [48]:
# we can access regular class methods from class objects
Widget.metamethod()

TracingMeta.metamethod(cls)
 cls = <class '__main__.Widget'>


In [49]:
# but we can't access these regular class mehods from the class instance objects
w = Widget()
w.metamethod()

AttributeError: 'Widget' object has no attribute 'metamethod'

#### Instance constructor
* when we create new class instance objects, we call the class constructor, which will call \_\_new\_\_ followed by \_\_init\_\_ in the class. This is the responsiblity of \_\_call\_\_ of metaclass
* assuming class Widget's metaclass is type, when we call w = Widget(), the \_\_call\_\_() method in type class will call the \_\_new\_\_(cls) method and \_\_init\_\_(). One implementation of \_\_call\_\_ is the following:

```python
class Widget(object, metaclass=type):
    
    def __new__(cls, *args, **kwargs):
        return type.__new__(cls)
    
    def __init__(self, *args, **kwargs):
        pass
```

```python
class type:
    def __call__(cls, *args, **kwargs):
        obj = cls.__new__(*args, **kwargs)
        
        obj.__init__(*args, **kwargs)
        return obj
```    
* note that the \_\_call\_\_() method is an instance method in the metaclass.
  + it obtain the cls (similar to self in class) in the metaclss after the \_\_new\_\_ method in the metaclass creates cls
  + it then use cls t call \_\_new\_\_() in the class, obtain the class instance object
  + then call the object's \_\_init\_\_() to configure the instance object